In [5]:
import netCDF4
import sys
from datetime import datetime, timedelta
import os
import requests
from urllib.parse import urlparse
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

sys.path.append('../src/')
from Biologging_Toolkit.auxiliary import Api_GPM

## Telechargement des données GPM depuis fichier .txt
Le code suivant permet de télécharger des données depuis le site GES DISC by NASA. Pour utiliser ce service, suivez les instructions ci-dessous :

- Création de compte :
    Rendez-vous sur le site GES DISC by NASA.
    Créez un compte en suivant les instructions fournies.

- Accès aux données :
    Une fois connecté, allez dans la section Download du site.

- Sélection des paramètres :
    Choisissez les dates, les coordonnées géographiques et les variables qui vous intéressent.

- Téléchargement des données :
    Après avoir sélectionné les paramètres, téléchargez le fichier .txt généré. Ce fichier contient toutes les URL nécessaires pour accéder aux données souhaitées.

Une fois le document .txt généré et téléchargé, vous pourrez spécifier le chemin d'accès vers ce document "file_path" ainsi que le dossier de téléchargement souhaité "output_dir" afin de telecharger l'ensemble des fichier GPM


### /!\ Pour pouvoir utiliser l'API, il faut vous identifier auprès de l'API. Pour ce faire, il est necessaire de créer un fichier à la racine de votre machine (Generalement "C:/users/user_name") . Créez un fichier qui s'appelle ".netrc" et copiez/collez la ligne suivante en remplaçant <username> et <password> par vos identifiants de connection GES DISC

machine urs.earthdata.nasa.gov login <username> password <password>


In [1]:
file_path = 'e:/Downloads/subset_GPM_3IMERGHHL_07_20250410_083755_.txt'
output_dir = 'e:/Downloads/GPM_data'
max_threads = 3

with open(file_path, 'r') as f:
    urls = [line.strip() for line in f if line.strip()]

os.makedirs(output_dir, exist_ok=True)
echec_list = []

def download_file(url):
    parsed_url = urlparse(url)
    filename = parsed_url.path.split('/')[-1]
    file_path = os.path.join(output_dir, filename)
    
    try:
        result = requests.get(url, stream=True, timeout=30)
        result.raise_for_status()
        with open(file_path, 'wb') as f_out:
            for chunk in result.iter_content(chunk_size=8192):
                f_out.write(chunk)
        return url, True, None
    except requests.exceptions.RequestException as e:
        return url, False, str(e)

results = []
with ThreadPoolExecutor(max_workers=max_threads) as executor:
    futures = {executor.submit(download_file, url): url for url in urls}
    for future in tqdm(as_completed(futures), total=len(futures), desc="Téléchargement des fichiers"):
        url, success, error = future.result()
        if not success:
            print(f"❌ Échec: {url}\n   ↳ {error}")
            echec_list.append(url)

print("✅ Tous les téléchargements terminés.")


Téléchargement des fichiers:   1%|          | 605/50535 [20:23<37:05:24,  2.67s/it]

❌ Échec: https://gpm1.gesdisc.eosdis.nasa.gov/opendap/GPM_L3/GPM_3IMERGHHL.07/2019/035/3B-HHR-L.MS.MRG.3IMERG.20190204-S143000-E145959.0870.V07B.HDF5.nc4?MWprecipitation[0:0][980:2750][239:559],MWobservationTime[0:0][980:2750][239:559],IRprecipitation[0:0][980:2750][239:559],MWprecipSource[0:0][980:2750][239:559],IRinfluence[0:0][980:2750][239:559],precipitationUncal[0:0][980:2750][239:559],precipitationQualityIndex[0:0][980:2750][239:559],lat_bnds[239:559][0:1],precipitation[0:0][980:2750][239:559],probabilityLiquidPrecipitation[0:0][980:2750][239:559],randomError[0:0][980:2750][239:559],time_bnds[0:0][0:1],lon_bnds[980:2750][0:1],time,lon[980:2750],lat[239:559],latv,nv,lonv
   ↳ HTTPSConnectionPool(host='gpm1.gesdisc.eosdis.nasa.gov', port=443): Read timed out. (read timeout=30)


Téléchargement des fichiers:   3%|▎         | 1455/50535 [49:37<42:04:08,  3.09s/it]

❌ Échec: https://gpm1.gesdisc.eosdis.nasa.gov/opendap/GPM_L3/GPM_3IMERGHHL.07/2019/053/3B-HHR-L.MS.MRG.3IMERG.20190222-S080000-E082959.0480.V07B.HDF5.nc4?MWprecipitation[0:0][980:2750][239:559],MWobservationTime[0:0][980:2750][239:559],IRprecipitation[0:0][980:2750][239:559],MWprecipSource[0:0][980:2750][239:559],IRinfluence[0:0][980:2750][239:559],precipitationUncal[0:0][980:2750][239:559],precipitationQualityIndex[0:0][980:2750][239:559],lat_bnds[239:559][0:1],precipitation[0:0][980:2750][239:559],probabilityLiquidPrecipitation[0:0][980:2750][239:559],randomError[0:0][980:2750][239:559],time_bnds[0:0][0:1],lon_bnds[980:2750][0:1],time,lon[980:2750],lat[239:559],latv,nv,lonv
   ↳ HTTPSConnectionPool(host='gpm1.gesdisc.eosdis.nasa.gov', port=443): Read timed out. (read timeout=30)


Téléchargement des fichiers:  10%|▉         | 4957/50535 [2:46:40<23:36:37,  1.86s/it]

...

❌ Échec: https://gpm1.gesdisc.eosdis.nasa.gov/opendap/GPM_L3/GPM_3IMERGHHL.07/2020/317/3B-HHR-L.MS.MRG.3IMERG.20201112-S033000-E035959.0210.V07B.HDF5.nc4?MWprecipitation[0:0][980:2750][239:559],MWobservationTime[0:0][980:2750][239:559],IRprecipitation[0:0][980:2750][239:559],MWprecipSource[0:0][980:2750][239:559],IRinfluence[0:0][980:2750][239:559],precipitationUncal[0:0][980:2750][239:559],precipitationQualityIndex[0:0][980:2750][239:559],lat_bnds[239:559][0:1],precipitation[0:0][980:2750][239:559],probabilityLiquidPrecipitation[0:0][980:2750][239:559],randomError[0:0][980:2750][239:559],time_bnds[0:0][0:1],lon_bnds[980:2750][0:1],time,lon[980:2750],lat[239:559],latv,nv,lonv
   ↳ 503 Server Error: Service Unavailable for url: https://gpm1.gesdisc.eosdis.nasa.gov/data-redirect?code=CKmc8NMaZiE6PG70bmu_iGRcrjriuXCp02Q1ArUno_ofvQLLTQNAvS2styM0iArM8Iz5Kw20icrPEOOVPrqr9COOvh6SyRJsSP-5aSl1AjOfgWSDeZ1t4-uB8qk-n1uD3xIkeZ6-rid4Uw&state=aHR0cHM6Ly9ncG0xLmdlc2Rpc2MuZW9zZGlzLm5hc2EuZ292L29wZW5kYX

Téléchargement des fichiers:  63%|██████▎   | 31713/50535 [18:20:40<6:48:38,  1.30s/it] 

❌ Échec: https://gpm1.gesdisc.eosdis.nasa.gov/opendap/GPM_L3/GPM_3IMERGHHL.07/2020/318/3B-HHR-L.MS.MRG.3IMERG.20201113-S213000-E215959.1290.V07B.HDF5.nc4?MWprecipitation[0:0][980:2750][239:559],MWobservationTime[0:0][980:2750][239:559],IRprecipitation[0:0][980:2750][239:559],MWprecipSource[0:0][980:2750][239:559],IRinfluence[0:0][980:2750][239:559],precipitationUncal[0:0][980:2750][239:559],precipitationQualityIndex[0:0][980:2750][239:559],lat_bnds[239:559][0:1],precipitation[0:0][980:2750][239:559],probabilityLiquidPrecipitation[0:0][980:2750][239:559],randomError[0:0][980:2750][239:559],time_bnds[0:0][0:1],lon_bnds[980:2750][0:1],time,lon[980:2750],lat[239:559],latv,nv,lonv
   ↳ HTTPSConnectionPool(host='gpm1.gesdisc.eosdis.nasa.gov', port=443): Max retries exceeded with url: /opendap/GPM_L3/GPM_3IMERGHHL.07/2020/318/3B-HHR-L.MS.MRG.3IMERG.20201113-S213000-E215959.1290.V07B.HDF5.nc4?MWprecipitation%5B0:0%5D%5B980:2750%5D%5B239:559%5D,MWobservationTime%5B0:0%5D%5B980:2750%5D%5B239:559

Téléchargement des fichiers: 100%|██████████| 50535/50535 [28:50:53<00:00,  2.06s/it]   

✅ Tous les téléchargements terminés.


## Gestion et télechargement des échecs

In [3]:
print(echec_list)
print(len(echec_list))

['https://gpm1.gesdisc.eosdis.nasa.gov/opendap/GPM_L3/GPM_3IMERGHHL.07/2019/035/3B-HHR-L.MS.MRG.3IMERG.20190204-S143000-E145959.0870.V07B.HDF5.nc4?MWprecipitation[0:0][980:2750][239:559],MWobservationTime[0:0][980:2750][239:559],IRprecipitation[0:0][980:2750][239:559],MWprecipSource[0:0][980:2750][239:559],IRinfluence[0:0][980:2750][239:559],precipitationUncal[0:0][980:2750][239:559],precipitationQualityIndex[0:0][980:2750][239:559],lat_bnds[239:559][0:1],precipitation[0:0][980:2750][239:559],probabilityLiquidPrecipitation[0:0][980:2750][239:559],randomError[0:0][980:2750][239:559],time_bnds[0:0][0:1],lon_bnds[980:2750][0:1],time,lon[980:2750],lat[239:559],latv,nv,lonv', 'https://gpm1.gesdisc.eosdis.nasa.gov/opendap/GPM_L3/GPM_3IMERGHHL.07/2019/053/3B-HHR-L.MS.MRG.3IMERG.20190222-S080000-E082959.0480.V07B.HDF5.nc4?MWprecipitation[0:0][980:2750][239:559],MWobservationTime[0:0][980:2750][239:559],IRprecipitation[0:0][980:2750][239:559],MWprecipSource[0:0][980:2750][239:559],IRinfluence[0

In [4]:
with ThreadPoolExecutor(max_workers=max_threads) as executor:
    futures = {executor.submit(download_file, url): url for url in echec_list}
    for future in tqdm(as_completed(futures), total=len(futures), desc="Téléchargement des fichiers"):
        url, success, error = future.result()
        if not success:
            print(f"❌ Échec: {url}\n   ↳ {error}")
            echec_list.append(url)

print("✅ Tous les téléchargements terminés.")

Téléchargement des fichiers: 100%|██████████| 21/21 [00:42<00:00,  2.03s/it]

✅ Tous les téléchargements terminés.


### Vérification que l'ensemble des données ont bien été telechargées

In [27]:
gpm_files = []

# dates souhaitées :  2017-10-06 to 2021-12-10
gpm_dates = Api_GPM.get_gpm_file_name(datetime(2017, 10, 6), datetime(2021, 12, 11))
print(len(gpm_dates), "files from 2017-10-06 to 2021-12-10")

# Ajout du reste du nom des fichiers
for dates in gpm_dates :
    gpm_files.append(f"3B-HHR-L.MS.MRG.3IMERG.{dates}.V07B.HDF5.nc4")

73297 files from 2017-10-06 to 2021-12-10


In [31]:
# Verification des dates des derniers fichiers telechargés
files_gpm_dwld = [f for f in os.listdir("e:/Downloads/GPM_data/") if not f.endswith('.pdf')]
files_gpm_dwld.sort()
print("Download files : ",files_gpm_dwld[-1])
print("File list      : ",gpm_files[-1])

# Verification de la présence de tous les fichiers dans le dossier de téléchargement
files_not_found = []
for file in tqdm(files_gpm_dwld):
    if file not in gpm_files:
        print(file, 'not in folder')
        files_not_found.append(file)
if len(files_not_found) > 0:
    print("Files not found : ", files_not_found)
else :
    print("All files found in the folder")

Download files :  3B-HHR-L.MS.MRG.3IMERG.20211210-S233000-E235959.1410.V07B.HDF5.nc4
File list      :  3B-HHR-L.MS.MRG.3IMERG.20211211-S000000-E002959.0000.V07B.HDF5.nc4


100%|██████████| 73295/73295 [00:48<00:00, 1502.96it/s]

All files found in the folder
